BoW Vektörü Elde Etme

In [2]:
# Gerekli kütüphaneleri yükleme
from sklearn.feature_extraction.text import CountVectorizer
import re
import pandas as pd

# 1. Dosyadan Metin Okuma Fonksiyonu
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# 2. Önişleme Fonksiyonu
def preprocess_text(text):
    # Küçük harfe çevirme
    text = text.lower()
    # Sayılar ve noktalama işaretlerini kaldırma
    text = re.sub(r'\d+', '', text)       # Sayıları kaldır
    text = re.sub(r'[^\w\s]', '', text)    # Noktalama işaretlerini kaldır
    # Durak kelimeler ve diğer gelişmiş işlemler eklenebilir
    return text

# 3. Bag-of-Words (BoW) Vektörlerini Oluşturma
def generate_bow_vectors(text):
    vectorizer = CountVectorizer()
    bow_matrix = vectorizer.fit_transform([text])  # Metni vektöre çevir
    feature_names = vectorizer.get_feature_names_out()
    bow_df = pd.DataFrame(bow_matrix.toarray(), columns=feature_names)
    return bow_df

# 4. Ana Program
file_path = 'dosyalar/odtu_derlemi.txt'  # İşlenecek dosyanın yolu
text = read_text_file(file_path)   # Metni dosyadan oku
preprocessed_text = preprocess_text(text)  # Metni ön işlemeden geçir
bow_df = generate_bow_vectors(preprocessed_text)  # BoW vektörlerini oluştur

In [3]:
# Her kelimenin sıklık sırasını ekleyerek sıralı liste oluşturma
def get_top_n_words_with_rank(bow_df, n=100):
    # Kelime frekanslarını hesapla
    word_frequencies = bow_df.sum(axis=0)
    # Frekansa göre azalan sırada sıralama ve ilk n kelimeyi al
    top_words = word_frequencies.sort_values(ascending=False).head(n)
    # Sıra numarası eklemek için DataFrame'e dönüştürme
    top_words_df = top_words.reset_index()
    top_words_df.columns = ['Kelime', 'Frekans']
    # Sıra numarasını ekleme
    top_words_df['Sıra'] = range(1, len(top_words_df) + 1)
    return top_words_df[['Sıra', 'Kelime', 'Frekans']]

# İlk 100 kelimeyi sıralı olarak elde etme
top_100_words_with_rank = get_top_n_words_with_rank(bow_df, 100)
print("Sıralı En Sık Kullanılan İlk 100 Kelime:")
print(top_100_words_with_rank.to_string(index=False))

Sıralı En Sık Kullanılan İlk 100 Kelime:
 Sıra     Kelime  Frekans
    1        bir    48507
    2         ve    34089
    3         bu    22776
    4         de    17075
    5         da    16725
    6       için    10860
    7        çok     7873
    8       gibi     7250
    9       daha     6827
   10        ile     6416
   11         ne     6369
   12        ama     6087
   13     olarak     5894
   14      sonra     5871
   15        her     5053
   16      kadar     5010
   17         en     4882
   18       olan     4695
   19       diye     4015
   20        var     3809
   21       dedi     3747
   22        ise     3724
   23        iki     3723
   24        ben     3711
   25         ki     3661
   26      değil     3504
   27      büyük     3411
   28         ya     3373
   29       yeni     3219
   30     içinde     3067
   31   olduğunu     2997
   32      zaman     2884
   33        ilk     2848
   34     olduğu     2836
   35       önce     2812
   36      yüzde     26

In [4]:
import os
import re
import xml.etree.ElementTree as ET
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# 1. Haber Klasöründen XML Dosyalarını Okuma ve Metni Çekme Fonksiyonu
def extract_text_from_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        # 'text' alanındaki metni çıkarma
        text_field = root.find(".//field[@name='text']")
        if text_field is not None:
            return text_field.text.strip()
        else:
            return ""
    except ET.ParseError as e:
        print(f"Hata: {file_path} XML dosyası düzgün değil. Hata: {e}")
        return ""

# 2. Önişleme Fonksiyonu
def preprocess_text(text):
    # Küçük harfe çevirme
    text = text.lower()
    # Sayılar ve noktalama işaretlerini kaldırma
    text = re.sub(r'\d+', '', text)       # Sayıları kaldır
    text = re.sub(r'[^\w\s]', '', text)    # Noktalama işaretlerini kaldır
    # Durak kelimeleri kaldırmak isterseniz buraya ekleyebilirsiniz
    return text

# 3. Belge ve Etiketleri Çekme
def load_documents_and_labels(base_dir):
    categories = os.listdir(base_dir)
    documents = []
    labels = []
    for category in categories:
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            for file_name in os.listdir(category_path):
                if file_name.endswith(".xml"):
                    file_path = os.path.join(category_path, file_name)
                    text = extract_text_from_xml(file_path)
                    processed_text = preprocess_text(text)
                    documents.append(processed_text)
                    labels.append(category)  # Her belgenin kategorisini etiket olarak kullan
    return documents, labels

# 4. Bag-of-Words (BoW) Vektörlerini Oluşturma
def generate_bow_vectors(documents):
    vectorizer = CountVectorizer()
    bow_matrix = vectorizer.fit_transform(documents)  # Belge koleksiyonunu BoW vektörlerine çevir
    feature_names = vectorizer.get_feature_names_out()
    bow_df = pd.DataFrame(bow_matrix.toarray(), columns=feature_names)
    return bow_df, vectorizer

# 5. Ana Program
base_dir = 'dosyalar/Milliyet-Collection-Mini'  # Veri klasörünün yolu
documents, labels = load_documents_and_labels(base_dir)  # Dosyaları yükle ve işlenmiş metinleri al

# 6. BoW Modeli Oluştur ve Her Belge için BoW Vektörlerini Al
bow_df, vectorizer = generate_bow_vectors(documents)

# Etiketleri ekleyerek belge vektörlerini birleştir
bow_df['Kategori'] = labels

# Sonuçları Görüntüleme
print("Bag-of-Words Vektörleri ve Kategoriler:")
print(bow_df)


Bag-of-Words Vektörleri ve Kategoriler:
     aatçi  ab  ababaya  abacı  abanozun  abant  abartıldığı  abc  abd  abdde  \
0        0   0        0      0         0      0            0    0    0      0   
1        0   0        0      0         0      0            0    0    0      0   
2        0   0        0      0         0      0            0    0    0      0   
3        0   0        0      0         0      0            0    0    0      0   
4        0   0        0      0         0      0            0    0    0      0   
..     ...  ..      ...    ...       ...    ...          ...  ...  ...    ...   
195      0   0        0      0         0      0            0    0    0      0   
196      0   0        0      0         0      0            0    0    0      0   
197      0   0        0      0         0      0            0    0    0      0   
198      0   0        0      0         0      0            0    0    0      0   
199      0   0        0      0         0      0            0    0    

Her bir haber dosyasına atanan BoW vektörlerini dosyaya yazdıralım

In [5]:
# BoW matrisini transpoze etme
bow_df_transposed = bow_df.transpose()

# TAB ile ayrılmış formatta dosyaya yazma
output_file = 'dosyalar/bow_matrix_transposed.txt'
bow_df_transposed.to_csv(output_file, sep='\t', header=False, index=True)

print(f"Transpoze edilmiş BoW matrisi '{output_file}' dosyasına yazıldı.")

Transpoze edilmiş BoW matrisi 'dosyalar/bow_matrix_transposed.txt' dosyasına yazıldı.


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 1. Özellikler (BoW vektörleri) ve Etiketleri Ayırma
X = bow_df.drop(columns=['Kategori'])
y = bow_df['Kategori']

# 2. Eğitim ve Test Setlerine Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Basit Makine Öğrenmesi Modelleri ile Eğitim ve Değerlendirme

# Modelleri tanımlama
models = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC(kernel='linear')
}

# Modelleri değerlendirme
for model_name, model in models.items():
    # Modeli eğitme
    model.fit(X_train, y_train)
    # Test verileri ile tahmin yapma
    y_pred = model.predict(X_test)
    # Sonuçları değerlendirme
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"{model_name} Sonuçları:")
    print(f"Doğruluk (Accuracy): {accuracy:.2f}")
    print(f"F1 Skoru: {f1:.2f}")
    print("Sınıflandırma Raporu:")
    print(classification_report(y_test, y_pred))
    print("\n" + "-"*50 + "\n")


Naive Bayes Sonuçları:
Doğruluk (Accuracy): 0.85
F1 Skoru: 0.85
Sınıflandırma Raporu:
              precision    recall  f1-score   support

     ekonomi       0.67      0.91      0.77        11
     magazin       0.86      0.90      0.88        20
     siyaset       0.92      0.80      0.86        15
        spor       1.00      0.79      0.88        14

    accuracy                           0.85        60
   macro avg       0.86      0.85      0.85        60
weighted avg       0.87      0.85      0.85        60


--------------------------------------------------

Logistic Regression Sonuçları:
Doğruluk (Accuracy): 0.87
F1 Skoru: 0.87
Sınıflandırma Raporu:
              precision    recall  f1-score   support

     ekonomi       0.82      0.82      0.82        11
     magazin       1.00      0.85      0.92        20
     siyaset       0.87      0.87      0.87        15
        spor       0.76      0.93      0.84        14

    accuracy                           0.87        60
   mac

Şimdi bir modelin verilen bir metin dosyasını hangi kategoriye dahil ettiğini görelim

In [7]:
# Test belgesinden metni çekme fonksiyonu
def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Önişleme fonksiyonu
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)       # Sayıları kaldır
    text = re.sub(r'[^\w\s]', '', text)    # Noktalama işaretlerini kaldır
    return text

# Yeni bir test .txt belgesini yükleyip işleyin
test_file_path = 'dosyalar/test2.txt'
test_text = read_text_file(test_file_path)
preprocessed_test_text = preprocess_text(test_text)

# Eğitim sırasında kullanılan vektörleştirici ile BoW vektörü oluşturma
test_vector = vectorizer.transform([preprocessed_test_text])

# Tahmin için en iyi modeli seçin (örneğin, en yüksek performans gösteren Naive Bayes)
predicted_category = models['Naive Bayes'].predict(test_vector)[0]

# Tahmin sonucunu ekrana yazdırma
print(f"Test belgesi tahmin edilen kategori: {predicted_category}")


Test belgesi tahmin edilen kategori: magazin


C:\Users\manue\anaconda3\envs\tensorflow_env_3-9\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


Şimdi de aynı işi kelime vektörleri ile yapalım

In [9]:
import os
import re
import xml.etree.ElementTree as ET
import gensim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd

# 1. Haber Klasöründen XML Dosyalarını Okuma ve Metni Çekme Fonksiyonu
def extract_text_from_xml(file_path):
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        # 'text' alanındaki metni çıkarma
        text_field = root.find(".//field[@name='text']")
        if text_field is not None:
            return text_field.text.strip()
        else:
            return ""
    except ET.ParseError as e:
        print(f"Hata: {file_path} XML dosyası düzgün değil. Hata: {e}")
        return ""

# 2. Önişleme Fonksiyonu
def preprocess_text(text):
    # Küçük harfe çevirme
    text = text.lower()
    # Sayılar ve noktalama işaretlerini kaldırma
    text = re.sub(r'\d+', '', text)       # Sayıları kaldır
    text = re.sub(r'[^\w\s]', '', text)    # Noktalama işaretlerini kaldır
    # Durak kelimeleri kaldırmak isterseniz buraya ekleyebilirsiniz
    return text

# 3. Belge ve Etiketleri Çekme
def load_documents_and_labels(base_dir):
    categories = os.listdir(base_dir)
    documents = []
    labels = []
    for category in categories:
        category_path = os.path.join(base_dir, category)
        if os.path.isdir(category_path):
            for file_name in os.listdir(category_path):
                if file_name.endswith(".xml"):
                    file_path = os.path.join(category_path, file_name)
                    text = extract_text_from_xml(file_path)
                    processed_text = preprocess_text(text)
                    documents.append(processed_text)
                    labels.append(category)  # Her belgenin kategorisini etiket olarak kullan
    return documents, labels

# Özel Formatlı Kelime Vektörlerini Yükleme Fonksiyonu (boyut uyumluluğunu kontrol ederek)
def load_custom_word_vectors(file_path):
    word_vectors = {}
    vector_size = None  # İlk geçerli vektör boyutunu belirlemek için
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split('\t')
            word = parts[0]
            vector = np.array([float(x) for x in parts[1:]])
            if vector_size is None:
                vector_size = len(vector)
            # Sadece doğru boyuttaki vektörleri ekle
            if len(vector) == vector_size:
                word_vectors[word] = vector

    # Sözlük formatındaki vektörleri KeyedVectors formatına dönüştürme
    keyed_vectors = gensim.models.KeyedVectors(vector_size=vector_size)
    keyed_vectors.add_vectors(list(word_vectors.keys()), list(word_vectors.values()))
    return keyed_vectors


# Yükleme işlemi
word_vectors = load_custom_word_vectors('dosyalar/cbow_word_vectors.txt') #sg_word_vectors.txt için de dene!

# 5. Her Belgenin Vektör Ortalamasını Hesaplama
def get_document_vector(text, model):
    words = text.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)  # Kelime vektörü bulunmazsa sıfır vektörü döner
    else:
        return np.mean(word_vectors, axis=0)

# Ana Program
base_dir = 'dosyalar/Milliyet-Collection-Mini'  # Veri klasörünün yolu
documents, labels = load_documents_and_labels(base_dir)  # Dosyaları yükle ve işlenmiş metinleri al

# 6. Her belge için vektör ortalamalarını hesaplayın
X = np.array([get_document_vector(doc, word_vectors) for doc in documents])
y = np.array(labels)

# 7. Eğitim ve Test Setlerine Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 8. Makine Öğrenmesi Modellerini Eğitme ve Değerlendirme

# Modelleri tanımlama
models = {
    'Naive Bayes (Gaussian)': GaussianNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC(kernel='linear')
}

# Modelleri değerlendirme
for model_name, model in models.items():
    # Modeli eğitme
    model.fit(X_train, y_train)
    # Test verileri ile tahmin yapma
    y_pred = model.predict(X_test)
    # Sonuçları değerlendirme
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"{model_name} Sonuçları:")
    print(f"Doğruluk (Accuracy): {accuracy:.2f}")
    print(f"F1 Skoru: {f1:.2f}")
    print("Sınıflandırma Raporu:")
    print(classification_report(y_test, y_pred))
    print("\n" + "-"*50 + "\n")


Naive Bayes (Gaussian) Sonuçları:
Doğruluk (Accuracy): 0.85
F1 Skoru: 0.85
Sınıflandırma Raporu:
              precision    recall  f1-score   support

     ekonomi       0.79      1.00      0.88        11
     magazin       0.84      0.80      0.82        20
     siyaset       0.87      0.87      0.87        15
        spor       0.92      0.79      0.85        14

    accuracy                           0.85        60
   macro avg       0.85      0.86      0.85        60
weighted avg       0.86      0.85      0.85        60


--------------------------------------------------

Logistic Regression Sonuçları:
Doğruluk (Accuracy): 0.85
F1 Skoru: 0.85
Sınıflandırma Raporu:
              precision    recall  f1-score   support

     ekonomi       0.77      0.91      0.83        11
     magazin       0.89      0.80      0.84        20
     siyaset       0.82      0.93      0.87        15
        spor       0.92      0.79      0.85        14

    accuracy                           0.85      